# OPEN ORDER NOTIFICATION

This python aims to create the files send to suppliers taking into account the prioritazion of the lines and the different criterias

## 1. Data Retrieve

Three reports are needed:
1. SAP HANA: Hana will be our base report since is the most updated one. In this case we apply the filters in the query
2. OOR: Report download from EMSBI, from this we get information as the Shipment address and the acknowledgment of lines done in SV portal and ARIBA
3. Contact list: Excel file of the supplier with the contacts and the names

In [5]:
spList = pd.read_csv('{}/oor/linesActioned.csv'.format(path))
spList

Title;"STATUS"
0     4581385363_5;
1    4581388204_14;
2     4581401936_8;
3     4580933718_2;
4     4581340149_9;
..              ...
352   4580992282_5;
353   4581408119_3;
354   4581383421_6;
355  4581360412_27;
356   4581405203_1;

[357 rows x 1 columns]

In [3]:

#import pyhdb
import pyodbc
import hdbcli
import pandas as pd
import pandas as pd
import numpy as np
import datetime
import os
import openpyxl as xl
import openpyxl as xl
from openpyxl.styles import Font, Color, PatternFill, Protection, Alignment, Border, Side
from openpyxl.worksheet.datavalidation import DataValidation
import math
from datetime import datetime

DEBUG = False

path="C:/Development/Automation-tools/SC-EMABot"
contact= pd.read_excel('{}/PlantContactList/HoustonPSCBotSupplierList.xlsx'.format(path),engine='openpyxl')
contact= contact[['OA Vendor', 'Email','Expeditor','Expeditor Assigned']]
#Cleaning the information from the OOR
oor = pd.read_csv('{}/oor/Operational Open Order Report PSC.csv'.format(path))
oor=oor[['PO Line Number','Acknowledged (Yes/No)','Order Date', 'Delivery/Dock Date (Per Incoterms)','Ship To Address, City', 'SLB Center']]
spList = pd.read_csv('{}/oor/linesActioned.csv'.format(path))



from hdbcli import dbapi
connection = dbapi.connect( address="sapw1pdb00.dir.slb.com", port=30015, user="LABOT", password="kC-mcLArs8b3E")

#connection = pyhdb.connect(host="sapw1pdb00.dir.slb.com",port=30015, user="LABOT",password="kC-mcLArs8b3E")
cursor = connection.cursor()
query="SELECT EBELN AS PO_Num, EBELP AS PO_Line, TXZ01 AS Material_Description, TO_CHAR(SLFDT,'MM/dd/yyyy') AS Delivery_Date, PPRO_LIFN2_OA AS OA_VENDOR, PPRO_LIFN2_PI AS VENDOR_NUMBER, TO_CHAR(PUVE_ERDAT,'MM/dd/yyyy') AS Creation_Date,"
query= query + "MATNR AS Material_Num, CM_QTY_DELIVER_LEFT AS Open_Quantity,VEND_NAME1 AS Vendor_Name,PURCHASE_ORDER_CONFIRMATIONS AS Acknowledgment, TO_CHAR(EINDT,'MM/dd/yyyy') AS Confirmed_del_date, TO_CHAR(ZZPERF_DATE,'MM/dd/yyyy') AS Performance_Date,"
query= query + "SOTX_TDLINE_F47 AS Action_Message_Priority,SOTX_TDLINE_F48 AS Action_Message_Status, ELIKZ AS GR_INDICATOR, EKGRP AS Purchasing_Group,BSART, EXT_RFX_NUMBER,EXT_RFX_ITEM, KNTTP, VEND_LAND1,CDOC_UDATE_RELEASE "
query= query + "FROM \"_SYS_BIC\".\"slb.bi.sup.sdi/SUP_STG_PO_END_TO_END_INTEGRATION\" "
query= query + "WHERE CM_QTY_DELIVER_LEFT > 0  "#Quantity to deliver greater than 0 (Open Quantity)
query= query + "AND UPPER(LOEKZ) <> UPPER('L') AND ELIKZ <> 'X'  "# Deleted lines filter out and GR indicator too
query= query + "AND EKGRP IN ('301','B02','B03','B04','B05','B06','B08') "# Filter of the purchasing Group
query= query + "AND WERKS IN ('1160','9036','9071','9052','9040','9119','9035');" #Filter for the plants
cursor.execute(query)
         
MyList=(cursor.fetchall())

#Sap hana query
df = pd.DataFrame(MyList)


df = df.rename(columns={
    0:'PO Num',
    1:'PO Line',
    2:'Material Description',
    3:'Delivery Date',
    4:'OA Vendor',
    5:'Vendor Number',
    6:'Creation Date',
    7:'Material Number',
    8:'Open Quantity',
    9:'Vendor Name',
    10:'Acknowledgment',
    11:'Confirmed Delivery Date',
    12:'Performance Date',
    13:'AMP',
    14:'AMS',
    15:'GR indicator',
    16:'Purch. Group',
    17:'Type PO',
    18:'Shopping Cart Numer',
    19:'Shopping Cart Line',
    20:'Account Category',
    21:'Vendor Country',
    22:'Release Date'
})



C:\Users\DOrtiz10\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,3,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Data Filtering:

Some of the filters applied to the report:
1. indirect lines
2. Lines not in OOR from EMS BI


In [51]:
def indirect(df):
    if df['Account Category'] =='K' or df['Account Category'] =='P':
        if len(str(df['Shopping Cart Numer']))>3 and (df['Purch. Group']=='301' or df['Purch. Group']=='B03' or df['Purch. Group']=='B02'):
            return 'Indirect'
        else:
            return 'Direct'
    else:
        return 'Direct'

df['Category']=df.apply(indirect,axis=1)

#Excluded Indirect Lines report:
excluded=df.loc[df['Category']=='Indirect']
df=df.loc[df['Category']!='Indirect']


#Creating the key of PO line
df['PO Line']=pd.to_numeric(df['PO Line'], errors='coerce')
df['PO-Line']= df['PO Num']+"-" +df['PO Line'].astype(str)

#Merging with the OOR information
df=pd.merge( left=df, right=oor, how='left',left_on='PO-Line', right_on='PO Line Number')

df['PO Line Number'] = df['PO Line Number'].fillna('notscope')
df = df[df['PO Line Number'] != 'notscope']


df['OA Vendor'] = df['OA Vendor'].astype(str)
contact['OA Vendor'] = contact['OA Vendor'].astype(str)
#Merging with the Contact list Info
df=pd.merge( left=df, right=contact, how='left',on='OA Vendor')
df['Email'] = df['Email'].fillna('notscope')
df = df[df['Email'] != 'notscope']


In [52]:
def internalFilter(df):
    if "P" in df['OA Vendor'] or "I" in df['OA Vendor']:
        return "Internal"
    else:
        return "External"
        
df['Internal']=df.apply(internalFilter, axis=1)
df = df[df['Internal'] != 'Internal']

# Prioritization
1. First we create the columns needed for the prioritize:

     a. Past due and Past due Aging
     
     b. Acknowledgment and Aknowledgment aging
     
     c. AMP
     
     d. Domestic-International
     
     

In [53]:
#Past Due
df["Delivery/Dock Date (Per Incoterms)"]=df["Delivery/Dock Date (Per Incoterms)"].str.split(" ",n=1,expand=True)[0]
df["DDD"] = pd.to_datetime(df["Delivery/Dock Date (Per Incoterms)"],
                                    format="%Y/%m/%d")



today = pd.datetime.now()
#ack
df["Order Date"]=df["Order Date"].str.split(" ",n=1,expand=True)[0]
df["PO Order Date"] = pd.to_datetime(df["Order Date"],
                                    format="%Y/%m/%d")


df['Ack Aging']= (today-df['PO Order Date']).dt.days

df=df[df["Ack Aging"]>8]
df=df[df["Acknowledged (Yes/No)"]=="No"]




C:\Users\DOrtiz10\AppData\Local\Temp/ipykernel_23896/4145677502.py:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime.now()


## Creating the files

1. Creation of the template file

2. Separation of the total lines by OA Vendor

3. Creation of excel file.

4. Repor with all lines 

In [54]:
print(df.columns)

Index(['PO Num', 'PO Line', 'Material Description', 'Delivery Date',
       'OA Vendor', 'Vendor Number', 'Creation Date', 'Material Number',
       'Open Quantity', 'Vendor Name', 'Acknowledgment',
       'Confirmed Delivery Date', 'Performance Date', 'AMP', 'AMS',
       'GR indicator', 'Purch. Group', 'Type PO', 'Shopping Cart Numer',
       'Shopping Cart Line', 'Account Category', 'Vendor Country',
       'Release Date', 'Category', 'PO-Line', 'PO Line Number',
       'Acknowledged (Yes/No)', 'Order Date',
       'Delivery/Dock Date (Per Incoterms)', 'Ship To Address, City',
       'SLB Center', 'Email', 'Expeditor', 'Expeditor Assigned', 'Internal',
       'DDD', 'PO Order Date', 'Ack Aging'],
      dtype='object')


In [55]:
dff=pd.DataFrame(df[['OA Vendor', 'Vendor Name', 'PO Num','PO Line', 'Material Number', 'Material Description', 'Open Quantity', 'Acknowledged (Yes/No)', 'SLB Center', 'Ship To Address, City', 'DDD','Expeditor','Email']])
dff=dff.rename(columns={
    'Ship To Address, City': 'Ship to address',
    'DDD': 'Delivery/Dock Date'
})

#Add blank columns
dff['Confirmed Delivery Date (MM/DD/YYYY)']=''
dff['Comments']=''
dff['Delivery/Dock Date']=dff['Delivery/Dock Date'].dt.strftime('%m/%d/%Y')

In [56]:
def excel_manipulation(ruta):
    wb = xl.load_workbook(ruta)
    ws = wb.active
    global temp_df

    l = len(temp_df)+1
    thin_border = Border(left=Side(style='thin'),
                         right=Side(style='thin'),
                         top=Side(style='thin'),
                         bottom=Side(style='thin')                    
                        )
    for row in ws.iter_rows():
            for cell in row:
                cell.alignment = Alignment(wrapText=False)
                cell.font = Font(name='Calibri', size=8)
                cell.border = thin_border
    #modifing the header style
    for i in ws[1]:
        i.font = Font(color="ffffff", name='Arial', size=9)

    a1 = ws['A1']
    b1 = ws['B1']
    c1 = ws['C1']
    d1 = ws['D1']
    e1 = ws['E1']
    f1 = ws['F1']
    g1 = ws['G1']
    h1 = ws['H1']
    i1 = ws['I1']
    j1 = ws['J1']
    k1 = ws['K1']
    l1 = ws['L1']
    m1 = ws['M1']



    a1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    b1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    c1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    d1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    e1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    f1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    g1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    h1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    i1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    j1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    k1.fill = PatternFill(fgColor="00003366", fill_type = "solid")
    l1.fill = PatternFill(fgColor="002eb05b", fill_type = "solid")
    m1.fill = PatternFill(fgColor="002eb05b", fill_type = "solid")



    #fit to the width
    col = ws.column_dimensions['A'].width = 10
    col = ws.column_dimensions['B'].width = 25
    col = ws.column_dimensions['C'].width = 12
    col = ws.column_dimensions['D'].width = 8
    col = ws.column_dimensions['E'].width = 18
    col = ws.column_dimensions['F'].width = 35
    col = ws.column_dimensions['G'].width = 12
    col = ws.column_dimensions['H'].width = 20
    col = ws.column_dimensions['I'].width = 20
    col = ws.column_dimensions['J'].width = 40
    col = ws.column_dimensions['K'].width = 30
    col = ws.column_dimensions['L'].width = 45
    col = ws.column_dimensions['M'].width = 45



    ws.row_dimensions[1].height = 15
    
    
    #Dropdown List and validation of date
    ds = DataValidation(type="custom",formula1='=AND(ISNUMBER(M2),LEFT(CELL("format",M2),1)="D")')
    ws.add_data_validation(ds)
    
    ds.add('M2:M{}'.format(l))
    
    #Adding dropdown list to the cell
    
    #Protecting the sheet
    
    ws.protection.sheet = True
    ws.protection.password = 'EM2021'

    starter = 0
    for cell in ws['m']:
        if starter == 0:
            pass
        else:
            cell.protection = Protection(locked=False)
            cell.number_format = "mm/dd/yyyy"
        starter+=1

    starter = 0
    for cell in ws['k']:
        if starter == 0:
            pass
        else:
            cell.protection = Protection(locked=False)
        starter+=1
        
    
    wb.save(filename=ruta)

In [57]:
tempSuccess=""
dbInsert=pd.DataFrame(columns=['OA Vendor','Success Insert'])
x=0
dfGrouped=dff.groupby('OA Vendor')
fpath=path+ '/oor/ackFiles'
for name, group in dfGrouped:
    temp_df= group.sort_values(by=['PO Num','PO Line'])
    temp_df=temp_df.drop(columns={'Expeditor','Email',})
    split_file_path = '{}/{}.xlsx'.format(fpath, name)
    temp_df.to_excel(split_file_path,index=False)
    excel_manipulation(split_file_path)
    
    #Creating the records for each supplier
    numVal=len(group)
    itera=math.ceil(numVal/1000)
    group=group.applymap(str)
    group.reset_index(inplace=True)
    for j in range(1,itera+1):
        query="INSERT INTO ""dbo"".""T_SCEMABOT_LOG"" ( PO_LINE,  VENDOR, PLANT, ACKNOWLEDGE, CREATION_DATE, STATUS,  EXPEDITOR, USER_EMAIL, RUN_NAME) VALUES "
        rowNum=0
        tempSuccess1=""
        if j == (itera):
            x=len(group)-((j-1)*1000)
        else:
            x=1000
        for m in range(0,x):
            i=m+(((j-1)*1000))
            if rowNum==0:
                tempSuccess1=query + "('" + str(group['PO Num'][i]) + "_"+ str(group['PO Line'][i])+ "'," + "'" + str(group['OA Vendor'][i]).replace("'", " ") + "',"+ "'" + str(group['SLB Center'][i]).replace("'", " ") + "'," + "'" + str(group['Acknowledged (Yes/No)'][i]) + "',"  + "CONVERT(date, GETDATE()),"  + "'Success',"  +"'" + group['Expeditor'][i] + "'," + "'" + group['Email'][i].replace("'", " ") + "','AGN'),"   
            else:
                tempSuccess1=tempSuccess1 + "('" + str(group['PO Num'][i]) + "_"+ str(group['PO Line'][i])+ "'," + "'" + str(group['OA Vendor'][i]).replace("'", " ") + "',"+ "'" +  str(group['SLB Center'][i]).replace("'", " ") + "'," + "'" + str(group['Acknowledged (Yes/No)'][i]) + "',"  + "CONVERT(date, GETDATE()),"  + "'Success',"  +"'" + group['Expeditor'][i] + "'," + "'" + group['Email'][i].replace("'", " ") + "','AGN'),"  
            rowNum+=1
        tempSuccess=tempSuccess1[:len(tempSuccess1)-1].replace("'None'", "NULL")
        dbInsert.loc[len(dbInsert.index)]=[name, tempSuccess]
        
dff.to_excel('{}/ZZEMABOT.xlsx'.format(fpath))


In [58]:
driver = 'SQL Server'
server='azr2680prdsql05.earaa2680.azr.slb.com'
database='sc-config'
Driver='ODBC Driver 17 for SQL Server'
username='scadmin'
password='Sc@dmin01'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

dbfailures=pd.DataFrame(columns=['OA Vendor','Insert'])


from datetime import datetime
for i, row in dbInsert.iterrows():
    try:
        cursor.execute(row['Success Insert'])
    except:
        dbfailures.loc[len(dbfailures.index)]=[row['OA Vendor'], row['Success Insert']]
    
    
cursor.commit()

if len(dbfailures)>1:
    dbfailures.to_excel('{}/InsertFailuresAGN.xlsx'.format(path))
    

print("Complete")

Complete
